## Clowder APIs

## Import libraries and setup utility function

We start by importing the rquired libraries, data and setting up some utility functions and variables that we will use below.

In [29]:
import pyclowder
import json
import os
import pandas as pd

import requests

# Function to download the IRIS dataset
def download_iris_dataset():
    # URL for the Iris dataset hosted by UCI Machine Learning Repository
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        with open("iris.csv", "wb") as f:
            f.write(response.content)
        print("The Iris dataset has been downloaded and saved as iris.csv.")
    else:
        print("Failed to download the dataset. Status code:", response.status_code)


CLOWDER_URL = "http://localhost:8000"

## Token Generation

In [30]:
user_login_json = {
    "email": "mohanar2@illinois.edu",
    "password": "password"
}
login_url = CLOWDER_URL + "/api/v2/login"
response = requests.post(login_url, json = user_login_json)
token = response.json()["token"]
print(response.json())

{'token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2dUVlQ0xOc1hTQXZUN1VDek1FRVk2VmI4ajJnY1RhWlFESUpnbnFGSHVJIn0.eyJleHAiOjE3MTY1NzcwNTgsImlhdCI6MTcxNjU3Njc1OCwianRpIjoiZjBjYmZlYzYtZjJlMS00NDg0LTg0YWMtNzFjMGMwZWNmNTNlIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo4MDgwL2tleWNsb2FrL3JlYWxtcy9jbG93ZGVyIiwic3ViIjoiZjg0Y2JjNmQtYzEzZC00MmVmLWFhN2MtMWQ4MmFjYzVhZWViIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xvd2RlcjItYmFja2VuZCIsInNlc3Npb25fc3RhdGUiOiIzOTgyN2EyNy1kMDA5LTRiY2YtOGIzYS1jZTk1MGM3YTI4OWQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDo4MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJkZWZhdWx0LXJvbGVzLWNsb3dkZXIiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIiwic2lkIjoiMzk4MjdhMjctZDAwOS00YmNmLThiM2EtY2U5NTBjN2EyODlkIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5hbWUiOiJWaXNtYXlhayBNb2hhbmFyYWphbiIsInByZWZlcnJlZF91c2VybmFtZSI6Im1vaGFuYXIyQGlsbGlub2lzLmVkdSIsImdpdmVuX25hbWUiOiJWaXNtYXlhayIsImZhbWlseV9uYW1lIjoiTW9oYW5hcmFqYW4iLCJlbWFpbCI6Im1vaGFuYXIyQGlsbGlub2lzL

## Creating Dataset

In [31]:
dataset_json = {
    "name": "Flower Dataset",
    "description": "Dataset for Flower Data",
    "status": "PRIVATE",
}
dataset_params = {
    "license_id": "CC BY"
}

token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICI2dUVlQ0xOc1hTQXZUN1VDek1FRVk2VmI4ajJnY1RhWlFESUpnbnFGSHVJIn0.eyJleHAiOjE3MTY1NzY5NjcsImlhdCI6MTcxNjU3NjY2NywiYXV0aF90aW1lIjoxNzE2NTc2Mzc1LCJqdGkiOiI5NzU3NTM1Mi04Y2FhLTRhMmUtODZhZC0wM2JkNDcwODhhZjkiLCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjgwODAva2V5Y2xvYWsvcmVhbG1zL2Nsb3dkZXIiLCJzdWIiOiJmODRjYmM2ZC1jMTNkLTQyZWYtYWE3Yy0xZDgyYWNjNWFlZWIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJjbG93ZGVyMi1iYWNrZW5kIiwic2Vzc2lvbl9zdGF0ZSI6IjdhMjJjZDFkLTQ3NDctNDAyNS1iNDk2LTUyYzNhOWUwZDhiYyIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwOi8vbG9jYWxob3N0OjgwMDAiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImRlZmF1bHQtcm9sZXMtY2xvd2RlciIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZW1haWwiLCJzaWQiOiI3YTIyY2QxZC00NzQ3LTQwMjUtYjQ5Ni01MmMzYTllMGQ4YmMiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwibmFtZSI6IlZpc21heWFrIE1vaGFuYXJhamFuIiwicHJlZmVycmVkX3VzZXJuYW1lIjoibW9oYW5hcjJAaWxsaW5vaXMuZWR1IiwiZ2l2ZW5fbmFtZSI6IlZpc21heWFrIiwiZmFtaWx5X25hbWUiOiJNb2hhbmFyYWphbiIsImVtYWlsIjoibW9oYW5hcjJAaWxsaW5vaXMuZWR1In0.AHqeZ79L8OUW2zTYEaDeUmuhHAeDMcqlgR8CE7ScGBRt-4gj8QFipbkiiMt0LYbMNi97dv0O_-iojrMtnlaX1OubnEwI_mMcoGckpXc5CzCoAQ_Qri2ZnpfnClDa9wPu-G_aGL-Sv4UpAglSTCbpBxsE99EiBubMb0T3NpP8p0k_lJTSvAZPJdMJ2sPpBo4BPUUIZq9JBXAZF7YPL9ZzUOoQKJvpNlA_7fHBHDjdSKRbVjP5cCNldjHJ70D2j1HM4JJUwMirNWZ8SFaJiny5a7NoZ2fa_JQkA3ZRrwGPHxGh6JThhu_F-a-pv2CDSytPCSU5DJxnslNJ4ePLVKDWpQ"
headers = {
    'Authorization': f'Bearer {token}'
}


dataset_url = CLOWDER_URL + "/api/v2/datasets"
response = requests.post(dataset_url, json = dataset_json, headers = headers, params = dataset_params)
print(response.json())

{'name': 'Flower Dataset', 'description': 'Dataset for Flower Data', 'status': 'PRIVATE', 'id': '6650e202fcd9057e97ae2d8a', 'creator': {'email': 'mohanar2@illinois.edu', 'first_name': 'Vismayak', 'last_name': 'Mohanarajan', 'id': '663a45d5b75ca83d17ac6564', 'admin': True, 'admin_mode': True}, 'created': '2024-05-24T18:52:50.457552', 'modified': '2024-05-24T18:52:50.457556', 'user_views': 0, 'downloads': 0, 'thumbnail_id': None, 'standard_license': True, 'license_id': 'CC BY'}
